# Amazon SageMaker Debugger - Reacting to Cloudwatch Events from Rules

*이 노트북은 [Amazon SageMaker Debugger - Reacting to Cloudwatch Events from Rules (영문 원본)](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/sagemaker-debugger/tensorflow_action_on_rule/tf-mnist-stop-training-job.ipynb) 의 한국어 번역입니다.*

[Amazon SageMaker](https://aws.amazon.com/sagemaker/) 는 머신 러닝 모델을 빌드(build), 학습(train) 및 호스팅(host)하기 위한 관리 플랫폼(managed platform)입니다. Amazon SageMaker Debugger는 거의 실시간으로 모델의 문제를 식별하고 감지하여 학습 중에 머신 러닝 모델을 디버깅할 수 있는 새로운 기능입니다.

이 노트북에서는 SageMaker built-in Rules을 사용하여 TensorFlow 프레임워크 상에서 Amazon SageMaker Debugger를 사용하는 방법을 수행해 보겠습니다.

## Amazon SageMaker Debugger는 어떻게 동작하나요?
Amazon SageMaker Debugger를 사용하면 학습 중에 loss 및 정확도(accuracy)와 같은 스칼라 값을 보는 것뿐만 아니라, 학습 중에 모든 텐서가 '그래프를 통해 흐르고 있습니다(flowing through the graph)'에 대한 완벽한 가시성(full visibility)을 제공합니다. 또한 규칙(rule)을 사용하여 거의 실시간으로 학습을 모니터링하고 학습 흐름(training flow)에 불일치가 감지되면 경고를 출력합니다.

### Concepts
* **Tensors(텐서)**: 학습 중 중간 지점(intermediate points)의 네트워크 상태를 나타냅니다.
* **Debug Hook(디버그 후크)**: Hook는 Amazon SageMaker Debugger가 학습 프로세스를 살펴보고 원하는 step 간격으로 요청된 텐서를 캡처합니다.
일부 프레임워크 컨테이너(TensorFlow 1.15, MXNet 1.6, PyTorch 1.3, XGBoost >= 0.90)들은 학습 스크립트 변경 없이 곧바로 Debugger가 활성화되며(`src/mnist_zerocodechange.py` 참조), 
다른 컨테이너의 경우 텐서 캡처 및 분석에 필요한 라이브러리 `smdebug`를 설치하고 학습 스크립트에 후크 콜백 함수를 추가하여 Debugger를 활성화할 수 있습니다(`src/mnist_byoc.py` 참조).
참고로, 본 워크샵에서는 학습 스크립트 변경이 필요 없는 TensorFlow 1.15를 사용합니다.
* **Rule(규칙)**: S3에 Hook로 캡처된 텐서를 가져와서 분석 후 이상점들(anomalies)이 발견되면 예외를 발생시키고 Amazon CloudWatch 이벤트를 트리거합니다. 여러 Rule들을 동시에 실행할 수 있습니다. 
빌트인(Built-in) Rule들은 2020년 2월 1일 기준 18개를 제공하고 있으며, 저장할 텐서/텐서 구성 방법 및 사용할 사용자 정의 Rule을 구성할 수 있는 고급 모드도 지원합니다.

상기 컨셉들을 염두에 두고 Amazon SageMaker Debugger의 전체 흐름을 이해해 보시죠.

### 학습 중 텐서 저장
디버그 후크(Debug hook)로 캡처된 텐서는 사용자가 지정한 S3 경로에 저장됩니다. 이 때, Amazon SageMaker Debugger릁 구성할 수 있는 두 가지 방법들이 있습니다.

#### 학습 스크립트 미변경 (With no changes to your training script)
Amazon SageMaker의 일부 빌트인 컨테이너들([Deep Learning Containers](https://docs.aws.amazon.com/sagemaker/latest/dg/pre-built-containers-frameworks-deep-learning.html))은 텐서를 저장하기 위해 학습 스크립트를 변경할 필요가 없습니다. Amazon SageMaker Debugger는 사용자가 지정한 방식으로 텐서를 저장하는 작업을 생성할 때, Amazon SageMaker SDK의 Tensorflow `Estimator`를 통해 제공한 설정(configuration)을 사용합니다. 
참고로, TensorFlow 프레임워크 상에서 Amazon SageMaker Debugger는 `tf.keras`, `tf.Estimator` 및 `tf.MonitoredSession` 인터페이스만 지원합니다. Debugger에 대한 자세한 설명은 [Amazon SageMaker Debugger with TensorFlow ](https://github.com/awslabs/sagemaker-debugger/tree/master/docs/tensorflow.md) 에서 확인할 수 있습니다.

#### 텐서를 저장하기 위한 스크립트 오케스트레이션 (Orchestrating your script to store tensors)
다른 컨테이너의 경우 여러분은 학습 스크립트를 몇 줄 변경해야 합니다. Amazon SageMaker Debugger는 이러한 텐서를 캡처하고 분석을 위해 저장할 수 있는 라이브러리 `smdebug`를 제공합니다. 이 라이브러리는 사용자 정의가 매우 용이하며(highly customizable), 다양한 설정 방법으로 여러분이 원하는 특정 텐서를 저장할 수 있습니다. 학습 스크립트에서 여러분이 선택한 프레임워크와 함께 Debugger 라이브러리를 사용하는 방법에 대한 자세한 내용은 [Developer Guide](https://github.com/awslabs/sagemaker-debugger/tree/master/docs)를 참조하세요. 참고로, `src/mnist_byoc` 폴더에 오케스트레이션된 예제 스크립트가 있습니다. 컨테이너에 `smdebug` 라이브러리가 설치되어 있는지 반드시 확인하세요.

### 텐서 분석

텐서가 저장되면, Amazon SageMaker Debugger에서 디버깅 규칙(***Rules***)을 실행하도록 구성할 수 있습니다. 매우 넓은 수준(very broad level)에서 규칙은 학습 중 특정 조건을 감지하는 데 사용되는 파이썬 코드입니다. 모델을 학습하는 데이터 과학자가 관심을 가질 수 있는 조건들 중 일부는 gradient가 너무 커지거나 작아지는지 모니터링하고 과적합(overfitting)을 감지하는 것입니다. Amazon Sagemaker Debugger에는 특정 퍼스트파티(1P) 규칙이 사전 패키지로 제공되며, 사용자는 Amazon Sagemaker Debugger API를 사용하여 자체 규칙을 작성할 수 있습니다. Amazon Sagemaker Debugger의 전체 API셋을 사용하여 Sagemaker 노트북과 같은 규칙 구성 외부에서 raw 텐서 데이터를 분석할 수도 있습니다. 관련 API에 대한 자세한 내용은 [Analysis Developer Guide](https://github.com/awslabs/sagemaker-debugger/blob/master/docs/api.md)를 참조하세요.

### Cloudwatch Events for Rules
학습 작업의 규칙(Rule) 상태(status) 변경은 CloudWatch Events를 트리거합니다. 이러한 이벤트는 디버거 규칙이 상태를 변경할 때마다 트리거하도록 CloudWatch 규칙(Amazon SageMaker 디버거 규칙과 다른)을 구성하여 수행할 수 있습니다. 이 노트북에서는 규칙이 트리거되고 상태가 `"IssuesFound"`인 경우, 학습 작업 상태 변경(Training Job State event) 이벤트를 학습 작업을 중지하는 lambda 함수로 보내도록 CloudWatch 규칙을 생성하는 방법을 살펴 보겠습니다.

#### Lambda Function

* AWS 콘솔에서 Lambda Management Console로 이동하세요.
* Create Function을 클릭하여 새 function을 생성하세요.
* 언어를 Python 3.7로 선택하고 Rule status 중 하나가 `"IssuesFound"`인 경우 학습 작업을 중지하기 위해 다음 샘플 코드를 붙여 넣으세요.

```python
import json
import boto3
import logging

def lambda_handler(event, context):
    training_job_name = event.get("detail").get("TrainingJobName")
    eval_statuses = event.get("detail").get("DebugRuleEvaluationStatuses", None)

    if eval_statuses is None or len(eval_statuses) == 0:
        logging.info("Couldn't find any debug rule statuses, skipping...")
        return {
            'statusCode': 200,
            'body': json.dumps('Nothing to do')
        }

    client = boto3.client('sagemaker')

    for status in eval_statuses:
        if status.get("RuleEvaluationStatus") == "IssuesFound":
            logging.info(
                'Evaluation of rule configuration {} resulted in "IssuesFound". '
                'Attempting to stop training job {}'.format(
                    status.get("RuleConfigurationName"), training_job_name
                )
            )
            try:
                client.stop_training_job(
                    TrainingJobName=training_job_name
                )
            except Exception as e:
                logging.error(
                    "Encountered error while trying to "
                    "stop training job {}: {}".format(
                        training_job_name, str(e)
                    )
                )
                raise e
    return None
```

* Lambda에 대한 신규 execution role을 생성하고,
* IAM 콘솔에서 역할(role)을 검색하고 "AmazonSageMakerFullAccess" 정책(policy)을 역할에 연결하세요. 이는 Lambda 함수의 코드가 학습 작업을 중지하는 데 필요합니다.

#### Create a CloudWatch Rule

* AWS 콘솔에서 CloudWatch로 이동하여 왼쪽 열에서 Rule을 선택하세요.
* Create Rule 버튼을 클릭하세요. 콘솔이 규칙 생성 페이지로 리다이렉션됩니다.
 * Service Name으로 "SageMaker"를 선택하세요.
 * Event Type으로 "SageMaker Training Job State Change"을 선택하세요.
* Targets에서 위에서 생성한 Lambda 함수를 선택하고 나머지는 그대로 둡니다.

In [1]:
!pip install --q --upgrade pip botocore awscli

In [2]:
!pip install --q smdebug matplotlib sagemaker

In [3]:
import boto3
import os
import sagemaker
from sagemaker.tensorflow import TensorFlow

In [4]:
from sagemaker.debugger import Rule, rule_configs

In [5]:
PATH = '/root/sagemaker-studio-workshop-kr/debugger'

In [6]:
# define the entrypoint script
entrypoint_script='{}/src/mnist_zerocodechange.py'.format(PATH)

# these hyperparameters ensure that vanishing gradient will trigger for our tensorflow mnist script
hyperparameters = {
    "num_epochs": "10",
    "lr": "10.00"
}

In [154]:
rules=[
    Rule.sagemaker(rule_configs.vanishing_gradient()), 
    Rule.sagemaker(rule_configs.loss_not_decreasing())
]

estimator = TensorFlow(
    role=sagemaker.get_execution_role(),
    base_job_name='smdebugger-demo-mnist-tensorflow',
    train_instance_count=1,
    train_instance_type='ml.m4.xlarge',
    entry_point=entrypoint_script,
    framework_version='1.15',
    train_volume_size=400,
    py_version='py3',
    train_max_run=3600,
    script_mode=True,
    hyperparameters=hyperparameters,
    ## New parameter
    rules = rules
)

In [155]:
# After calling fit, SageMaker will spin off 1 training job and 1 rule job for you
# The rule evaluation status(es) will be visible in the training logs
# at regular intervals
# wait=False makes this a fire and forget function. To stream the logs in the notebook leave this out

estimator.fit(wait=False)

## Monitoring

SageMaker는 estimator에서 지정한 대로 각 SageMaker 규칙 (`VanishingGradient` 및 `LossNotDecreasing`)에 대해 하나씩 규칙 평가 작업을 시작했습니다.
`VanishingGradient`가 활성화될 수 있도록 학습 스크립트의 하이퍼 파라미터를 조정했으므로, `VanishingGradient`에 대한 `RuleEvaluationStatus`가 `IssuesFound`로 변경되면 `TrainingJobStatus`가 `Stopped`로 표시됩니다.

In [191]:
# rule job summary gives you the summary of the rule evaluations. You might have to run it over 
# a few times before you start to see all values populated/changing
estimator.latest_training_job.rule_job_summary()

[{'RuleConfigurationName': 'VanishingGradient',
  'RuleEvaluationJobArn': 'arn:aws:sagemaker:us-east-2:072677473360:processing-job/smdebugger-demo-mnist-tens-vanishinggradient-e23301a8',
  'RuleEvaluationStatus': 'IssuesFound',
  'StatusDetails': 'RuleEvaluationConditionMet: Evaluation of the rule VanishingGradient at step 500 resulted in the condition being met\n',
  'LastModifiedTime': datetime.datetime(2019, 12, 1, 7, 20, 55, 495000, tzinfo=tzlocal())},
 {'RuleConfigurationName': 'LossNotDecreasing',
  'RuleEvaluationJobArn': 'arn:aws:sagemaker:us-east-2:072677473360:processing-job/smdebugger-demo-mnist-tens-lossnotdecreasing-27ee2da1',
  'RuleEvaluationStatus': 'InProgress',
  'LastModifiedTime': datetime.datetime(2019, 12, 1, 7, 20, 55, 495000, tzinfo=tzlocal())}]

In [194]:
# This utility gives the link to monitor the CW event
def _get_rule_job_name(training_job_name, rule_configuration_name, rule_job_arn):
        """Helper function to get the rule job name"""
        return "{}-{}-{}".format(
            training_job_name[:26], rule_configuration_name[:26], rule_job_arn[-8:]
        )
    
def _get_cw_url_for_rule_job(rule_job_name, region):
    return "https://{}.console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/ProcessingJobs;prefix={};streamFilter=typeLogStreamPrefix".format(region, region, rule_job_name)


def get_rule_jobs_cw_urls(estimator):
    region = boto3.Session().region_name
    training_job = estimator.latest_training_job
    training_job_name = training_job.describe()["TrainingJobName"]
    rule_eval_statuses = training_job.describe()["DebugRuleEvaluationStatuses"]
    
    result={}
    for status in rule_eval_statuses:
        if status.get("RuleEvaluationJobArn", None) is not None:
            rule_job_name = _get_rule_job_name(training_job_name, status["RuleConfigurationName"], status["RuleEvaluationJobArn"])
            result[status["RuleConfigurationName"]] = _get_cw_url_for_rule_job(rule_job_name, region)
    return result

get_rule_jobs_cw_urls(estimator)

{'VanishingGradient': 'https://us-east-2.console.aws.amazon.com/cloudwatch/home?region=us-east-2#logStream:group=/aws/sagemaker/ProcessingJobs;prefix=smdebugger-demo-mnist-tens-VanishingGradient-e23301a8;streamFilter=typeLogStreamPrefix',
 'LossNotDecreasing': 'https://us-east-2.console.aws.amazon.com/cloudwatch/home?region=us-east-2#logStream:group=/aws/sagemaker/ProcessingJobs;prefix=smdebugger-demo-mnist-tens-LossNotDecreasing-27ee2da1;streamFilter=typeLogStreamPrefix'}

`VanishingGradient`가 `IssuesFound`를 출력할 때까지, 위의 두 Code cell들을 반복 실행해 주세요.
`IssuesFound` 출력 시, 학습 작업에 대한 `TrainingJobStatus`가 `Stopped`으로 변경되는 것을 확인할 수 있습니다.

In [193]:
estimator.latest_training_job.describe()["TrainingJobStatus"]

'Stopped'

## Result

이 노트북은 학습 작업에 CloudWatch 이벤트를 사용하여 규칙 평가 상태 변경에 대한 조치를 수행하는 방법을 보여주었습니다. [GitHub Documentation](https://github.com/awslabs/sagemaker-debugger)에서 Amazon SageMaker 디버거에 대해 자세히 알아보세요.